In [43]:
import os
import pickle
from ultralytics import YOLO
import cv2
import numpy as np
from moviepy.editor import VideoFileClip, AudioFileClip, VideoClip
import subprocess
import os

def process_video(video_path: str) -> tuple[str, str]:
    """
    Process video through ASD pipeline.
    
    Args:
        video_path: Path to input video
        
    Returns:
        tuple: Path to scores and tracks pickle files
    """
    # Run ASD script
    os.system(f"bash ./run_asd_for_custom_video_in_local.sh {video_path}")
    
    # Get case ID and output directory
    case_id = os.path.basename(video_path).split('.mp4')[0]
    print(50*'-')
    print(case_id)
    print(50*'-')
    output_dir = f"./output/{case_id}"
    
    return f"{output_dir}/scores.pckl", f"{output_dir}/tracks.pckl"

def analyze_scene_and_scores(csv_path: str, scores_path: str) -> tuple[bool, float]:
    """
    Analyze if video has single scene and get mean scores.
    
    Args:
        csv_path: Path to scenes CSV
        scores_path: Path to scores pickle file
        
    Returns:
        tuple: (has_single_scene, mean_score)
    """
    # Check if single scene
    with open(csv_path, 'r') as f:
        has_single_scene = sum(1 for line in f) - 2 == 1
        
    # Get mean scores for frames 45-55
    with open(scores_path, 'rb') as f:
        scores = pickle.load(f)
    mean_score = np.mean(scores[0][45:55])
    
    return has_single_scene, mean_score

def get_bbox_center(tracks_path: str, frame_num: int = 50) -> tuple[float, float]:
    """
    Get center coordinates of bbox at specified frame.
    
    Args:
        tracks_path: Path to tracks pickle file
        frame_num: Frame number to analyze
        
    Returns:
        tuple: (center_x, center_y)
    """
    with open(tracks_path, 'rb') as f:
        tracks = pickle.load(f)
    
    # Get bbox coordinates for frame 50
    frame_idx = np.where(tracks[0]['track']['frame'] == frame_num)[0][0]
    bbox = tracks[0]['track']['bbox'][frame_idx]
    center_x = (bbox[0] + bbox[2]) / 2
    center_y = (bbox[1] + bbox[3]) / 2
    
    return center_x, center_y

def create_tracked_person_video(video_path: str, results: list, track_id: int, output_path: str, keep_audio: bool = True) -> None:
    """
    Creates a video with black background except for the tracked person's bounding box region.
    The bounding box is expanded by 5% on each side.
    
    Args:
        video_path: Path to the input video file
        results: List of detection results for all frames
        track_id: Track ID of the person to highlight
        output_path: Path where the output video will be saved
        keep_audio: Whether to keep the original audio in output video
        
    Returns:
        None
    """
    print("Starting video creation process...")
    
    # Open the video
    cap = cv2.VideoCapture(video_path)
    print(f"Opened video file: {video_path}")
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    print(f"Video properties - Width: {width}, Height: {height}, FPS: {fps}")
    
    # Create absolute paths
    abs_output_path = os.path.abspath(output_path)
    # Remove .mp4 extension if present to avoid double extension
    abs_output_path = abs_output_path.replace('.mp4', '')
    temp_output = abs_output_path + '_temp.mp4'
    final_output = abs_output_path + '.mp4'
    print(f"Temporary output path: {temp_output}")
    
    # Create VideoWriter object with more compatible codec
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using MP4V codec instead of H.264
    out = cv2.VideoWriter(temp_output, fourcc, fps, (width, height))
    
    if not out.isOpened():
        print("First codec failed, trying alternative...")
        # Try alternative codec if first attempt fails
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(temp_output, fourcc, fps, (width, height))
        if not out.isOpened():
            raise RuntimeError(f"Failed to create output video file at {temp_output}")
    
    print("Processing frames...")
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Create black frame
        black_frame = np.zeros_like(frame)
            
        # Get detection results for current frame
        if frame_idx < len(results):
            frame_results = results[frame_idx]
            boxes = frame_results.boxes.xyxy
            ids = frame_results.boxes.id
            
            # Find tracked person and copy their region from original frame
            for box, id in zip(boxes, ids):
                if int(id) == track_id:
                    x1, y1, x2, y2 = map(int, box)
                    
                    # Calculate 5% of width and height for expansion
                    w_expand = int((x2 - x1) * 0.05)
                    h_expand = int((y2 - y1) * 0.05)
                    
                    # Expand box by 5% each side, but stay within frame bounds
                    x1 = max(0, x1 - w_expand)
                    y1 = max(0, y1 - h_expand)
                    x2 = min(width, x2 + w_expand)
                    y2 = min(height, y2 + h_expand)
                    
                    # Copy the person region from original frame to black frame
                    black_frame[y1:y2, x1:x2] = frame[y1:y2, x1:x2]
        
        # Write frame
        out.write(black_frame)
        frame_idx += 1
        
    print(f"Processed {frame_idx} frames")
    
    # Release everything
    cap.release()
    out.release()

    if keep_audio:
        # Verify temp file exists
        print("OUTPUT:", 50*'-')
        print(temp_output)
        print(50*'-')
        print("ORIGINAL:", 50*'-')
        print(video_path)
        print(50*'-')
        print("FINAL OUTPUT:", 50*'-')
        print(final_output)
        print(50*'-')
        if not os.path.exists(temp_output):
            raise FileNotFoundError(f"Temporary video file not found at {temp_output}")
            
        # Extract audio from original video
        audio_path = os.path.join(os.path.dirname(temp_output), "temp_audio.aac")
        extract_audio_cmd = f'ffmpeg -i "{video_path}" -vn -acodec copy "{audio_path}" -y '
        print(extract_audio_cmd)        
        subprocess.run(extract_audio_cmd, shell=True, check=True)
            
        # Combine temp video with extracted audio
        combine_cmd = f'ffmpeg -i "{temp_output}" -i "{audio_path}" -c:v h264 -c:a aac "{final_output}" -y'
        print(combine_cmd)

        subprocess.run(combine_cmd, shell=True, check=True)
        
        # Clean up temp audio file
        # os.remove(audio_path)
            
    else:
        # Just rename temp file to final output
        os.rename(temp_output, final_output)


    
def get_track_id_at_point(frame_num: int, x: int, y: int, results: list) -> int | None:
    """
    Get the track ID of a person at a specific point in a frame.
    
    Args:
        frame_num: Frame number to check
        x: X coordinate of the point
        y: Y coordinate of the point 
        results: List of detection results for all frames
        
    Returns:
        Track ID if a person is found at the point, None otherwise
    """
    # Get frame results
    frame_results = results[frame_num]

    # Get all bounding boxes and IDs for this frame
    boxes = frame_results.boxes.xyxy
    track_ids = frame_results.boxes.id

    # Check each box if it contains the point
    for box, track_id in zip(boxes, track_ids):
        x1, y1, x2, y2 = box
        if (x1 <= x <= x2) and (y1 <= y <= y2):
            return int(track_id)
            
    return None

def main(video_path: str, output_dir: str = "./output"):
    """
    Main orchestration function.
    
    Args:
        video_path: Path to input video
    """
    # Step 1: Process through ASD
    scores_path, tracks_path = process_video(video_path)
    
    # Step 2: Analyze scene and scores
    case_id = os.path.basename(video_path).split('.mp4')[0]
    print(50*'-')
    print(case_id)
    print(50*'-')
    csv_path = f"./output/{case_id}/{case_id}.csv"
    
    has_single_scene, mean_score = analyze_scene_and_scores(csv_path, scores_path)
    print(f"Has single scene: {has_single_scene}")
    print(f"Mean score (frames 45-55): {mean_score}")
    
    if has_single_scene and mean_score > 0:
        # Step 3: Get bbox center
        center_x, center_y = get_bbox_center(tracks_path)
        print(f"Center point at frame 50: ({center_x:.2f}, {center_y:.2f})")

        model = YOLO("yolo11n.pt")
        results = model.track(source=video_path, conf=0.5, iou=0.5, show=True, save=True, save_dir="output", save_txt=True, save_json=True)
        track_id = get_track_id_at_point(50, center_x, center_y, results)
        output_path = f"./output/{case_id}/{case_id}_tracked.mp4"
        
        # Step 4: Track person and create video
        create_tracked_person_video(video_path, results, track_id, output_path, keep_audio=True)

        # if output video exists, move to output folder
        if os.path.exists(output_path):
            os.rename(output_path, f"{output_dir}/{os.path.basename(output_path)}")
    else:
        print("Video doesn't meet criteria (single scene with positive mean score)")



In [ ]:
def process_folder(input_dir: str, output_dir: str = "./salida") -> None:
    """
    Process all video files in a folder by calling main() on each.
    
    Args:
        input_dir: Path to folder containing video files
        output_dir: Path to output directory for processed videos
    """
    # Create output dir if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Get all video files
    video_files = [f for f in os.listdir(input_dir) 
                   if f.lower().endswith(('.mp4', '.avi', '.mov'))]
    
    for video_file in video_files:
        video_path = os.path.join(input_dir, video_file)
        print(50*'-')
        print(f"Processing: {video_file}")
        print(50*'-')
        try:
            main(video_path, output_dir=output_dir)
        except Exception as e:
            print(f"Error processing {video_file}: {str(e)}")
            continue

# Example usage
input_folder = "/home/raul/miscosas/repos/TalkNet-ASD/tmp"
process_folder(input_folder, output_dir="./salida")


In [44]:
!mkdir -p salida
# Example usage
print(50*'-')
print("2017-07-03_1300_US_KNBC_Today_in_LA_1769.859_1774.034_yesterday.mp4")
print(50*'-')
video_path = "/home/raul/miscosas/repos/TalkNet-ASD/tmp/2017-07-03_1300_US_KNBC_Today_in_LA_1769.859_1774.034_yesterday.mp4"
main(video_path, output_dir="./salida")

print(50*'-')
print("2019-06-03_0200_ES_24h_Telediario_2a_Edición_1998.278-2002.639_Nadal.mp4")
print(50*'-')
video_path = "/home/raul/miscosas/repos/TalkNet-ASD/tmp/2019-06-03_0200_ES_24h_Telediario_2a_Edición_1998.278-2002.639_Nadal.mp4"
main(video_path, output_dir="./salida")

print(50*'-')
print("2019-06-04_0200_ES_24h_Telediario_2a_Edición_384.789-389.19_Nadal.mp4")
print(50*'-')
video_path = "/home/raul/miscosas/repos/TalkNet-ASD/tmp/2019-06-04_0200_ES_24h_Telediario_2a_Edición_384.789-389.19_Nadal.mp4"
main(video_path, output_dir="./salida")

print(50*'-')
print("2019-06-04_0200_ES_24h_Telediario_2a_Edición_384.789-389.19_Nadal.mp4")
print(50*'-')
video_path = "/home/raul/miscosas/repos/TalkNet-ASD/tmp/2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal.mp4"
main(video_path, output_dir="./salida")

--------------------------------------------------
2017-07-03_1300_US_KNBC_Today_in_LA_1769.859_1774.034_yesterday.mp4
--------------------------------------------------
**************************************************2017-07-03_1300_US_KNBC_Today_in_LA_1769.859_1774.034_yesterday**************************************************
Processing video with following parameters:
CASE_ID = 2017-07-03_1300_US_KNBC_Today_in_LA_1769.859_1774.034_yesterday
VIDEO_PATH = /home/raul/miscosas/repos/TalkNet-ASD/tmp/2017-07-03_1300_US_KNBC_Today_in_LA_1769.859_1774.034_yesterday.mp4
SAVE_TO = ./output/2017-07-03_1300_US_KNBC_Today_in_LA_1769.859_1774.034_yesterday
TEMP_DIR = /tmp/talknet_processing
PATH_TO_SCENES = ./output/2017-07-03_1300_US_KNBC_Today_in_LA_1769.859_1774.034_yesterday/2017-07-03_1300_US_KNBC_Today_in_LA_1769.859_1774.034_yesterday.csv
FPS = 25
THREADS = 10
DEVICE = cuda
[PySceneDetect] PySceneDetect 0.6.5
[PySceneDetect] Detecting scenes...
[PySceneDetect] Processed 126 frames in 0

./run_asd_for_custom_video_in_local.sh: line 2: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
  Detected: 0 | Progress: 100%|██████████| 126/126 [00:00<00:00, 4392.82frames/s]


2017-07-03_1300_US_KNBC_Today_in_LA_1769.859_1774.034_yesterday
VIDEO PATH = /home/raul/miscosas/repos/TalkNet-ASD/tmp/2017-07-03_1300_US_KNBC_Today_in_LA_1769.859_1774.034_yesterday.mp4
SAVE TO ./output/2017-07-03_1300_US_KNBC_Today_in_LA_1769.859_1774.034_yesterday
File exists! Exiting...
--------------------------------------------------
2017-07-03_1300_US_KNBC_Today_in_LA_1769.859_1774.034_yesterday
--------------------------------------------------
--------------------------------------------------
2017-07-03_1300_US_KNBC_Today_in_LA_1769.859_1774.034_yesterday
--------------------------------------------------
Has single scene: True
Mean score (frames 45-55): 0.6699999965727329
Center point at frame 50: (413.74, 70.32)
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()
The DISPLAY environment variable isn't set.


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or lo

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

[PySceneDetect] PySceneDetect 0.6.5
[PySceneDetect] Detecting scenes...
[PySceneDetect] Processed 131 frames in 0.1 seconds (average 2435.19 FPS).
[PySceneDetect] Detected 2 scenes, average shot length 2.6 seconds.
[PySceneDetect] Writing scene list to CSV file:
  ./output/2019-06-03_0200_ES_24h_Telediario_2a_Edición_1998.278-2002.639_Nadal/2019-06-03_0200_ES_24h_Telediario_2a_Edición_1998.278-2002.639_Nadal.csv
2019-06-03_0200_ES_24h_Telediario_2a_Edición_1998.278-2002.639_Nadal
VIDEO PATH = /home/raul/miscosas/repos/TalkNet-ASD/tmp/2019-06-03_0200_ES_24h_Telediario_2a_Edición_1998.278-2002.639_Nadal.mp4
SAVE TO ./output/2019-06-03_0200_ES_24h_Telediario_2a_Edición_1998.278-2002.639_Nadal
File exists! Exiting...
--------------------------------------------------
2019-06-03_0200_ES_24h_Telediario_2a_Edición_1998.278-2002.639_Nadal
--------------------------------------------------
--------------------------------------------------
2019-06-03_0200_ES_24h_Telediario_2a_Edición_199

...

  Detected: 0 | Progress: 100%|██████████| 132/132 [00:00<00:00, 2725.65frames/s]
./run_asd_for_custom_video_in_local.sh: line 2: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


[PySceneDetect] PySceneDetect 0.6.5
[PySceneDetect] Detecting scenes...
[PySceneDetect] Processed 106 frames in 0.0 seconds (average 2964.79 FPS).
[PySceneDetect] Detected 1 scenes, average shot length 4.2 seconds.
[PySceneDetect] Writing scene list to CSV file:
  ./output/2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal/2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal.csv
2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal
VIDEO PATH = /home/raul/miscosas/repos/TalkNet-ASD/tmp/2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal.mp4
SAVE TO ./output/2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal
File exists! Exiting...
--------------------------------------------------
2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal
--------------------------------------------------
--------------------------------------------------
2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal
--------------

  Detected: 0 | Progress: 100%|██████████| 106/106 [00:00<00:00, 3117.89frames/s]


video 1/1 (frame 5/106) /home/raul/miscosas/repos/TalkNet-ASD/tmp/2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal.mp4: 512x640 1 person, 1 tie, 2.6ms
video 1/1 (frame 6/106) /home/raul/miscosas/repos/TalkNet-ASD/tmp/2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal.mp4: 512x640 1 person, 1 tie, 2.6ms
video 1/1 (frame 7/106) /home/raul/miscosas/repos/TalkNet-ASD/tmp/2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal.mp4: 512x640 1 person, 1 tie, 2.5ms
video 1/1 (frame 8/106) /home/raul/miscosas/repos/TalkNet-ASD/tmp/2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal.mp4: 512x640 1 person, 1 tie, 2.6ms
video 1/1 (frame 9/106) /home/raul/miscosas/repos/TalkNet-ASD/tmp/2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal.mp4: 512x640 1 person, 1 tie, 2.7ms
video 1/1 (frame 10/106) /home/raul/miscosas/repos/TalkNet-ASD/tmp/2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal.mp4: 512x640 1 person, 1 tie, 2.6

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [8]:
# Extract frame 50 using ffmpeg
video_path = "/home/raul/miscosas/repos/TalkNet-ASD/tmp/2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal.mp4"
output_frame = "frame_50.jpg"

# Use ffmpeg to extract frame 50 (note: ffmpeg is 0-based indexing)
!ffmpeg -y -i "{video_path}" -vf "select=eq(n\,49)" -vframes 1 "{output_frame}"



ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [19]:
video_path = "/home/raul/miscosas/repos/TalkNet-ASD/tmp/2019-06-27_0305_ES_24h_La_noche_en_24_horas_1294.79-1298.3_Nadal.mp4"
jpg_path = "/home/raul/miscosas/repos/TalkNet-ASD/frame_50.jpg"
from ultralytics import SAM

model = SAM("sam2_b.pt")

#model.info()

# model(source=jpg_path, points=[313, 110])
model(source=jpg_path, show=True, save=True, classes=[1])



WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()
The DISPLAY environment variable isn't set.

image 1/1 /home/raul/miscosas/repos/TalkNet-ASD/frame_50.jpg: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1 13, 1 14, 1323.0ms
Speed: 2.4ms preprocess, 1323.0ms inference, 0.2ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to runs/segment/predict4


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '10', 11: '11', 12: '12', 13: '13', 14: '14'}
 obb: None
 orig_img: array([[[ 0,  1,  0],
         [ 0,  1,  0],
         [ 0,  1,  0],
         ...,
         [23,  4,  1],
         [18,  0,  0],
         [22,  3,  0]],
 
        [[ 0,  1,  0],
         [ 0,  1,  0],
         [ 0,  1,  0],
         ...,
         [18,  0,  0],
         [18,  0,  0],
         [19,  0,  0]],
 
        [[ 0,  1,  0],
         [ 0,  1,  0],
         [ 0,  1,  0],
         ...,
         [25,  8,  5],
         [16,  0,  0],
         [16,  0,  0]],
 
        ...,
 
        [[ 2,  0,  0],
         [ 2,  0,  0],
         [ 2,  0,  0],
         ...,
         [ 6,  0, 12],
         [ 5,  0,  7],
         [ 5,  0,  3]],
 
        [[ 2,  0,  0],
       